***GENERATED CODE FOR hotelcla PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Hotel', 'transformation_label': 'String Indexer'}], 'feature': 'Hotel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'City Hotel', 'max': 'Resort Hotel', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Hotel'}, {'feature_label': 'Hotel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Hotel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Arrival_Month', 'transformation_label': 'String Indexer'}], 'feature': 'Arrival_Month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'August', 'max': 'September', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Arrival_Month'}, {'feature_label': 'Arrival_Month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Arrival_Month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Children', 'transformation_label': 'String Indexer'}], 'feature': 'Children', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '0', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Children'}, {'feature_label': 'Children', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Children')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Meal', 'transformation_label': 'String Indexer'}], 'feature': 'Meal', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'BB', 'max': 'BB', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Meal'}, {'feature_label': 'Meal', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Meal')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'PRT', 'max': 'PRT', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Market_segment', 'transformation_label': 'String Indexer'}], 'feature': 'Market_segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Direct', 'max': 'Online TA', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Market_segment'}, {'feature_label': 'Market_segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Market_segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Distribution_Channel', 'transformation_label': 'String Indexer'}], 'feature': 'Distribution_Channel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Corporate', 'max': 'TA/TO', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Distribution_Channel'}, {'feature_label': 'Distribution_Channel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Distribution_Channel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Room_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Room_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'H', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Room_Type'}, {'feature_label': 'Room_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Room_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Deposit_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Deposit_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No Deposit', 'max': 'Non Refund', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Deposit_Type'}, {'feature_label': 'Deposit_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Deposit_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Agent_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Agent_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '86.45', 'stddev': '104.0', 'min': '11', 'max': 'NULL', 'missing': '0', 'distinct': '36'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Agent_ID'}, {'feature_label': 'Agent_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Agent_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Customer_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Contract', 'max': 'Transient-Party', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer_Type'}, {'feature_label': 'Customer_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reservation_Status', 'transformation_label': 'String Indexer'}], 'feature': 'Reservation_Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Canceled', 'max': 'No-Show', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reservation_Status'}, {'feature_label': 'Reservation_Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reservation_Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reservation_Status_Date', 'transformation_label': 'String Indexer'}], 'feature': 'Reservation_Status_Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2015', 'max': '11/18/2014', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reservation_Status_Date'}, {'feature_label': 'Reservation_Status_Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reservation_Status_Date')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run hotelclaHooks.ipynb
try:
	#sourcePreExecutionHook()

	hotelbookingstrainaa = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/hotel bookings trainAA.csv', 'filename': 'hotel bookings trainAA.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(hotelbookingstrainaa)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run hotelclaHooks.ipynb
try:
	#transformationPreExecutionHook()

	hotelclaautofe = TransformationMain.run(hotelbookingstrainaa,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Hotel", "transformation_label": "String Indexer"}], "feature": "Hotel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "City Hotel", "max": "Resort Hotel", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Hotel"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Arrival_Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2016.13", "stddev": "0.83", "min": "2015", "max": "2017", "missing": "0"}, "updatedLabel": "Arrival_Year"}, {"transformationsData": [{"feature_label": "Arrival_Month", "transformation_label": "String Indexer"}], "feature": "Arrival_Month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "August", "max": "September", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Arrival_Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Arrival_Day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "17.69", "stddev": "5.51", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "Arrival_Day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Arrival_Week", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "12.04", "stddev": "7.66", "min": "6", "max": "53", "missing": "0"}, "updatedLabel": "Arrival_Week"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weekend_Nights_Stay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.2", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "Weekend_Nights_Stay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Week_Nights_Stay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.02", "stddev": "0.39", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "Week_Nights_Stay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Adults", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.0", "stddev": "0.0", "min": "2", "max": "2", "missing": "0"}, "updatedLabel": "Adults"}, {"transformationsData": [{"feature_label": "Children", "transformation_label": "String Indexer"}], "feature": "Children", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Children"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Babies", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.13", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Babies"}, {"transformationsData": [{"feature_label": "Meal", "transformation_label": "String Indexer"}], "feature": "Meal", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "BB", "max": "BB", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Meal"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "PRT", "max": "PRT", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"feature_label": "Market_segment", "transformation_label": "String Indexer"}], "feature": "Market_segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Direct", "max": "Online TA", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Market_segment"}, {"transformationsData": [{"feature_label": "Distribution_Channel", "transformation_label": "String Indexer"}], "feature": "Distribution_Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Corporate", "max": "TA/TO", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Distribution_Channel"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Previous_Cancellations", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Previous_Cancellations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Previous_none_canceled", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Previous_none_canceled"}, {"transformationsData": [{"feature_label": "Room_Type", "transformation_label": "String Indexer"}], "feature": "Room_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "H", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Room_Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Booking_Changes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.2", "stddev": "0.62", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "Booking_Changes"}, {"transformationsData": [{"feature_label": "Deposit_Type", "transformation_label": "String Indexer"}], "feature": "Deposit_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No Deposit", "max": "Non Refund", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Deposit_Type"}, {"transformationsData": [{"feature_label": "Agent_ID", "transformation_label": "String Indexer"}], "feature": "Agent_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "86.45", "stddev": "104.0", "min": "11", "max": "NULL", "missing": "0", "distinct": "36"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Agent_ID"}, {"transformationsData": [{"feature_label": "Customer_Type", "transformation_label": "String Indexer"}], "feature": "Customer_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Contract", "max": "Transient-Party", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Average_Daily_Rate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "62.16", "stddev": "3.31", "min": "0.0", "max": "90.95", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Average_Daily_Rate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Special_Request", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.38", "stddev": "0.62", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Special_Request"}, {"transformationsData": [{"feature_label": "Reservation_Status", "transformation_label": "String Indexer"}], "feature": "Reservation_Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Canceled", "max": "No-Show", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reservation_Status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Is_canceled", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "Is_canceled"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lead_time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "55.87", "stddev": "64.15", "min": "0", "max": "345", "missing": "0"}, "updatedLabel": "lead_time"}, {"transformationsData": [{"feature_label": "Reservation_Status_Date", "transformation_label": "String Indexer"}], "feature": "Reservation_Status_Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2015", "max": "11/18/2014", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reservation_Status_Date"}]}))

	#transformationPostExecutionHook(hotelclaautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run hotelclaHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(hotelclaautofe, ["Arrival_Year", "Arrival_Day", "Arrival_Week", "Weekend_Nights_Stay", "Week_Nights_Stay", "Adults", "Babies", "Previous_Cancellations", "Previous_none_canceled", "Booking_Changes", "Average_Daily_Rate", "Special_Request", "lead_time", "Hotel_stringindexer", "Arrival_Month_stringindexer", "Children_stringindexer", "Meal_stringindexer", "Country_stringindexer", "Market_segment_stringindexer", "Distribution_Channel_stringindexer", "Room_Type_stringindexer", "Deposit_Type_stringindexer", "Agent_ID_stringindexer", "Customer_Type_stringindexer", "Reservation_Status_stringindexer", "Reservation_Status_Date_stringindexer"], "Is_canceled")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

